# Capstone Journal

**5/15/2025**:   Class meetup. Read, set, go!

**5/19/2025**:   Reviewed requirements, created project directory, base files, and drafted a vision board.

![Management Cycle Flowchart](notebook/management-cycle.png)

- NOTE: I want to give the CSV data some context by generating a new fake csv that reflects a developers finances.
- ACTION: Created `csv_faker.py` to generate 500,000 transaction records and saved them to `developer_transactions.csv`.

In [6]:
# Task 1: Loading Transactions from a CSV File

# Parse data with datetime.strptime
# Make amount negative for 'debit'
# Convert transaction_id and customer_id to integers
# Create dictionary with all fields
# Add to transactions 
# Catch FileNotFoundError and ValueError

# NOTE: These functions will all go into a `FinanceUtils` class inside the utils.py file

import csv
from datetime import datetime

def load_transactions(filename='financial_transactions.csv'):
    """Load transactions from a CSV file into a list of dictionaries."""

    transactions = []
    with open(filename, mode='r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Convert transaction ID to integer
            row['transaction_id'] = int(row['transaction_id'])
            # Convert customer ID to integer
            row['customer_id'] = int(row['customer_id'])
            # Convert date string to datetime object
            row['date'] = datetime.strptime(row['date'], '%Y-%m-%d').date()
            # Convert amount to float
            row['amount'] = float(row['amount'])
            # Make amount negative for 'debit'
            if row['type'] == 'debit':
                row['amount'] = -row['amount']
            transactions.append(row)

    return transactions

# Test: Print the first 3 transactions
print("Loaded transactions:")
transactions = load_transactions(filename='developer_transactions.csv')
for transaction in range(3):
    print(transactions[transaction])

print(f"Total transactions loaded: {len(transactions)}")

Loaded transactions:
{'transaction_id': 1, 'date': datetime.date(2021, 11, 19), 'customer_id': 719, 'amount': 363.2, 'type': 'transfer', 'description': 'Payment to friend for dinner'}
{'transaction_id': 2, 'date': datetime.date(2024, 4, 16), 'customer_id': 674, 'amount': 456.13, 'type': 'transfer', 'description': 'Reimbursement for shared purchase'}
{'transaction_id': 3, 'date': datetime.date(2017, 4, 13), 'customer_id': 651, 'amount': 273.71, 'type': 'transfer', 'description': 'Transfer to investment account'}
Total transactions loaded: 500000


In [ ]:
# Class method:

import csv
from datetime import datetime
import logging
import os

class FinanceUtils:
    """Class to manage financial transactions with CRUD operations and analysis."""

    # Class constructor

    def load_transactions(self, filename='financial_transactions.csv'):
        """
        Load transactions from a CSV file into self.transactions.
        
        Args:
            filename (str): Path to the CSV file.
            
        Returns:
            bool: True if loading succeeds, False otherwise.
        """
        self.transactions = []
        required_columns = {'transaction_id', 'date', 'customer_id', 'amount', 'type', 'description'}

        try:
            with open(filename, mode='r', encoding='utf-8') as file:
                reader = csv.DictReader(file)

                # Check required columns
                if not required_columns.issubset(reader.fieldnames):
                    missing = required_columns - set(reader.fieldnames)
                    logging.error(f"Missing columns in CSV: {missing}")
                    print(f"Missing columns in CSV: {missing}")
                    return False
                
                for row_num, row in enumerate(reader, start=2):
                    try:
                        # Validate transaction_id
                        try:
                            transaction_id = int(row['transaction_id'])
                        except ValueError:
                            logging.error(f"Row {row_num}: Invalid transaction_id '{row['transaction_id']}'")
                            continue

                        # Validate date
                        date_str = row['date'].strip()
                        try:
                            date_obj = datetime.strptime(date_str, '%Y-%m-%d').date()
                        except ValueError:
                            logging.error(f"Row {row_num}: Invalid date format '{date_str}'")
                            continue

                        # Validate customer_id
                        try:
                            customer_id = int(row['customer_id'])
                        except ValueError:
                            logging.error(f"Row {row_num}: Invalid customer_id '{row['customer_id']}'")
                            continue

                        # Validate amount
                        try:
                            amount = float(row['amount'])
                            if amount < 0:
                                logging.error(f"Row {row_num}: Negative amount '{amount}'")
                                continue

                        except ValueError:
                            logging.error(f"Row {row_num}: Invalid amount '{row['amount']}'")
                            continue

                        # Validate type
                        transaction_type = row['type'].strip().lower()
                        if transaction_type not in {'credit', 'debit', 'transfer'}:
                            logging.error(f"Row {row_num}: Invalid transaction type '{transaction_type}'")
                            continue

                        # Adjust amount for debit
                        if transaction_type == 'debit':
                            amount = -amount

                        # Validate description
                        description = row.get('description')
                        if description is None or not str(description).strip():
                            logging.error(f"Row {row_num}: Empty description")
                            continue
                        description = str(description).strip()

                        # Create transaction dictionary
                        transaction = {
                            'transaction_id': transaction_id,
                            'date': date_obj,
                            'customer_id': customer_id,
                            'amount': amount,
                            'type': transaction_type,
                            'description': description
                        }
                        self.transactions.append(transaction)

                    except KeyError as e:
                        logging.error(f"Row {row_num}: Missing column {e}")
                        continue

                print(f"Loaded {len(self.transactions)} transactions from '{filename}'.")

                # Create a backup of the original file and save it with a timestamp to /snapshots
                if not os.path.exists('snapshots'):
                    os.makedirs('snapshots')

                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                backup_filename = os.path.join('snapshots', f'backup_{timestamp}.csv')
                try:
                    with open(filename, 'rb') as src_file, open(backup_filename, 'wb') as dst_file:
                        dst_file.write(src_file.read())
                    print(f"Backup created: '{backup_filename}'")
                except Exception as e:
                    logging.error(f"Failed to create backup: {e}")

                return True
            
        except FileNotFoundError:
            logging.error(f"File '{filename}' not found.")
            print(f"File '{filename}' not found.")
            return False
        
        except csv.Error:
            logging.error(f"Malformed CSV file '{filename}'.")
            print(f"Error reading CSV file '{filename}'.")
            return False
        
        except IOError as e:
            logging.error(f"IO error reading {filename}: {e}")
            print(f"Error: IO error reading file: {e}")
            return False
        
# *initial, unmodified class method

## Testing

✅ Skips rows with invalid fields and prints a warning.  
✅ Loads and counts transactions accurately.  
✅ Checks required columns  
✅ Validates transaction_id  
✅ Validates date  
✅ Validates customer_id  
✅ Validates amount  
✅ Validates type  
✅ Validates description  

✅ Backup CSV snapshots  
✅ Handle backup errors  

**5/21/2025**: Starting Task 2 and implementing CRUD features.  

**Goals**:  
- Reject invalid transaction types
- Add an option to filter by types when viewing table
- Suggest customer IDs from existing transactions
- Format dates as "Oct 26, 2020" 

Sticking with class methods for my finance utilities and pulling in a library for formatting pretty tables.

I'll write these methods and then test the terminal interface for usability and readability. Colors? Hm.

In [ ]:
# Example functions

def add_transaction(transactions):
    """Add a new transaction from user input."""
    # Prompt for date, customer_id, amount, type, description
    # Validate date, amount, type
    # Generate new transaction_id
    # Create dictionary and append
    pass


def view_transactions(transactions):
    """Display transactions in a table."""
    # Print header
    # Loop through transactions
    # Format each row
    pass

In [ ]:
# Class methods

# Using the `tabulate` library for better table formatting
from tabulate import tabulate

def add_transaction(self):
    print("\nAdd New Transaction (enter 'cancel' to abort)")

    # Date input
    while True:
        date_input = input("Enter date (YYYY-MM-DD): ").strip()
        if date_input.lower() == 'cancel':
            print("Transaction addition cancelled.")
            return False
        try:
            date_obj = datetime.strptime(date_input, '%Y-%m-%d').date()
            break
        except ValueError:
            logging.error(f"Invalid date format: {date_input}")
            print("Invalid date format. Please enter in YYYY-MM-DD format.")

    # Customer ID input with suggestions
    customer_ids = sorted(set(t['customer_id'] for t in self.transactions))
    if customer_ids:
        print(f"Valid customer IDs: {', '.join(map(str, customer_ids[:10]))}{'...' if len(customer_ids) > 10 else ''}")
    while True:
        customer_input = input("Enter customer ID (integer): ").strip()
        if customer_input.lower() == 'cancel':
            print("Transaction cancelled.")
            return False
        try:
            customer_id = int(customer_input)
            break
        except ValueError:
            logging.error(f"Invalid customer ID input: {customer_input}")
            print("Error: Customer ID must be an integer. Please try again.")

    # Amount input
    while True:
        amount_input = input("Enter amount (positive number): ").strip()
        if amount_input.lower() == 'cancel':
            print("Transaction cancelled.")
            return False
        try:
            amount = float(amount_input)
            if amount <= 0:
                logging.error(f"Non-positive amount input: {amount_input}")
                print("Error: Amount must be positive. Please try again.")
                continue
            break
        except ValueError:
            logging.error(f"Invalid amount input: {amount_input}")
            print("Error: Amount must be a number. Please try again.")

    # Type input
    valid_types = {'credit', 'debit', 'transfer'}
    while True:
        type_input = input("Enter type (credit/debit/transfer): ").strip().lower()
        if type_input.lower() == 'cancel':
            print("Transaction cancelled.")
            return False
        if type_input not in valid_types:
            logging.error(f"Invalid transaction type input: {type_input}")
            print(f"Error: Type must be one of {', '.join(valid_types)}. Please try again.")
            continue
        break

    # Adjust amount for debit
    if type_input == 'debit':
        amount = -amount

    # Description input
    while True:
        description = input("Enter description: (non-empty): ").strip()
        if description.lower() == 'cancel':
            print("Transaction cancelled.")
            return False
        if not description:
            logging.error("Empty description input")
            print("Error: Description cannot be empty. Please try again.")
            continue
        break

    # Generate new transaction ID
    transaction_id = max((t[transaction_id] for t in self.transactions), default=0) + 1

    # Create and append transaction
    transaction = {
        'transaction_id': transaction_id,
        'date': date_obj,
        'customer_id': customer_id,
        'amount': amount,
        'type': type_input,
        'description': description
    }
    self.transactions.append(transaction)
    print(f"Transaction {transaction} added successfully!")
    return True

def view_transactions(self, filter_type=None):
    if not self.transactions:
        print("No transactions to display.")
        return False
    
    # Apply filter
    valid_types = {'credit', 'debit', 'transfer'}
    if filter_type and filter_type.lower() not in valid_types:
        logging.error(f"Invalid filter type: {filter_type}")
        print(f"Error: Filter type must be one of {', '.join(valid_types)} or empty.")
        return False
    
    transactions = (
        [t for t in self.transactions if t['type'] == filter_type.lower()]
        if filter_type else self.transactions
    )

    if not transactions:
        print(f"No {filter_type} transactions found." if filter_type else "No transactions found.")
        return False
    
    # Prepare table data
    table = [
        [
            t['transaction_id'],
            t['date'].strftime('%b %d, %Y'),
            t['customer_id'],
            f"${t['amount']:,.2f}",
            t['type'].capitalize(),
            t['description'][:30] + ('...' if len(t['description']) > 30 else '')
        ]
        for t in transactions
    ]

    headers = ['ID', 'Date', 'Customer', 'Amount', 'Type', 'Description']
    print(f"\n{'Filtered' if filter_type else 'All'} Transactions:")
    print(tabulate(table, headers=headers, tablefmt='grid', stralign='left'))
    return True

**Notes**:  
Should I add pagination to the table results?  
Users can enter a negative customer ID when adding transactions; should always be positive.

**Tests**:  
✅ Test Invalid Inputs (Date, Customer ID, Amount, Type, Description)  
✅ Test Type Filters 

**Optional Tweaks**:  
- Paginate with navigation prompts
- Added year filter
- Validate `transaction_id` uniqueness
- Enforce positive `customer_id`: Reject negative values and log errors

**More Tests**:  
✅ Test Type/Year Filters  
✅ Test Non-Positive customer_id  
✅ Test Pagination Navigation  
✅ Test Transaction Views w/ Filters  
✅ Test Empty CSV

**5/22/2025**: Starting Task 3 and continuing CRUD features.  

**Example Code**  

```python
def update_transaction(transactions):
    """Update a transaction’s details."""
    # Show transactions with numbers
    # Ask user to pick a number
    # Ask which field to change
    # Update field
    pass

def delete_transaction(transactions):
    """Delete a transaction."""
    # Show transactions with numbers
    # Ask user to pick a number
    # Confirm and remove
    pass
```  

**Goals**:  
- Use enumerate for transaction numbers
- Validate transaction number input
- Confirm deletions with "Are you sure?"
- Prevent invalid type updates in update_transaction
- Show transaction details before deletion
- Allow updating multiple fields at once
- Add a cancel option for updates/deletions

In [ ]:
# Class method
def _get_transaction_by_id(self, transaction_id):
    """Helper method to find a transaction by its ID."""
    for t in self.transactions:
        if t['transaction_id'] == transaction_id:
            return t
    return None

def update_transaction(self):
    if not self.transactions:
        print("No transactions to update.")
        return False
    
    print("\nUpdate Transaction (enter 'cancel' to abort)")
    while True:
        id_input = input("Enter transaction ID (e.g., 123): ").strip()
        if id_input.lower() == 'cancel':
            print("Update cancelled.")
            return False
        try:
            transaction_id = int(id_input)
            transaction = self._get_transaction_by_id(transaction_id)
            if not transaction:
                logging.error(f"Transaction ID {transaction_id} not found.")
                print(f"Error: Transaction ID {transaction_id} not found. Try again.")
                continue
            break
        except ValueError:
            logging.error(f"Invalid transaction ID input: {id_input}")
            print("Error: Transaction ID must be an integer. Please try again.")

    # Display current transaction
    print(f"\nUpdating Transaction {transaction_id}:")
    print(f"  Current: {transaction['date'].strftime('%Y-%m-%d')}, Customer {transaction['customer_id']}, "
            f"${abs(transaction['amount']):,.2f} {transaction['type'].capitalize()}, {transaction['description']}")
    print("Enter new values (press Enter to keep current, 'cancel' to abort)")

    # Date input
    while True:
        date_input = input(f"New date [{transaction['date'].strftime('%Y-%m-%d')}]: ").strip()
        if date_input.lower() == 'cancel':
            print("Update cancelled.")
            return False
        if not date_input:
            date_obj = transaction['date']
            break
        try:
            date_obj = datetime.strptime(date_input, '%Y-%m-%d').date()
            break
        except ValueError:
            logging.error(f"Invalid date input: '{date_input}'")
            print("Error: Date must be in YYYY-MM-DD format (e.g., 2020-10-26). Try again.")

    # Customer ID input
    customer_ids = sorted(set(t['customer_id'] for t in self.transactions if t['customer_id'] > 0))
    if customer_ids:
        print(f"Valid customer IDs: {', '.join(map(str, customer_ids[:10]))}{'...' if len(customer_ids) > 10 else ''}")
    while True:
        customer_input = input(f"New customer ID [{transaction['customer_id']}]: ").strip()
        if customer_input.lower() == 'cancel':
            print("Update cancelled.")
            return False
        if not customer_input:
            customer_id = transaction['customer_id']
            break
        try:
            customer_id = int(customer_input)
            if customer_id <= 0:
                logging.error(f"Non-positive customer ID input: '{customer_id}'")
                print("Error: Customer ID must be a positive integer. Try again.")
                continue
            break
        except ValueError:
            logging.error(f"Invalid customer ID input: {customer_input}")
            print("Error: Customer ID must be a positive integer. Please try again.")

    # Amount input
    while True:
        amount_input = input(f"New amount [{abs(transaction['amount']):,.2f}]: ").strip()
        if amount_input.lower() == 'cancel':
            print("Update cancelled.")
            return False
        if not amount_input:
            amount = abs(transaction['amount'])
            break
        try:
            amount = float(amount_input)
            if amount <= 0:
                logging.error(f"Non-positive amount input: '{amount}'")
                print("Error: Amount must be positive. Try again.")
                continue
            break
        except ValueError:
            logging.error(f"Invalid amount input: '{amount_input}'")
            print("Error: Amount must be a number. Try again.")

    # Type input
    valid_types = {'credit', 'debit', 'transfer'}
    while True:
        type_input = input(f"New type [{transaction['type']}]: ").strip().lower()
        if type_input.lower() == 'cancel':
            print("Update cancelled.")
            return False
        if not type_input:
            transaction_type = transaction['type']
            break
        if type_input not in valid_types:
            logging.error(f"Invalid transaction type input: '{type_input}'")
            print(f"Error: Type must be one of {', '.join(valid_types)}. Try again.")
            continue
        break

    # Adjust amount for debit
    if type_input == 'debit':
        amount = -amount

    # Description input
    while True:
        description = input(f"New description [{transaction['description']}]: ").strip()
        if description.lower() == 'cancel':
            print("Update cancelled.")
            return False
        if not description:
            description = transaction['description']
            break
        if not description.strip():
            logging.error("Empty description input")
            print("Error: Description cannot be empty. Try again.")
            continue
        description = description.strip()
        break

    # Update transaction
    transaction.update({
        'date': date_obj,
        'customer_id': customer_id,
        'amount': amount,
        'type': type_input,
        'description': description
    })
    print(f"Transaction {transaction_id} updated successfully!")
    return True

def delete_transaction(self):
    """
    Prompt user to delete a transaction by ID.
        
    Returns:
        bool: True if transaction is deleted, False if cancelled or invalid.
    """
    if not self.transactions:
        print("No transactions to delete.")
        return False

    print("\nDelete Transaction (enter 'cancel' to abort)")
    while True:
        id_input = input("Enter transaction ID (e.g., 123): ").strip()
        if id_input.lower() == 'cancel':
            print("Deletion cancelled.")
            return False
        try:
            transaction_id = int(id_input)
            transaction = self._get_transaction_by_id(transaction_id)
            if not transaction:
                logging.error(f"Transaction ID not found: '{transaction_id}'")
                print(f"Error: Transaction ID {transaction_id} not found. Try again.")
                continue
            break
        except ValueError:
            logging.error(f"Invalid transaction ID input: '{id_input}'")
            print("Error: Transaction ID must be an integer. Try again.")

    # Display transaction
    print(f"\nTransaction to delete (ID {transaction_id}):")
    print(f"  {transaction['date'].strftime('%Y-%m-%d')}, Customer {transaction['customer_id']}, "
            f"${abs(transaction['amount']):,.2f} {transaction['type'].capitalize()}, {transaction['description']}")

    # Confirm deletion
    while True:
        confirm = input("Are you sure? (y/n): ").strip().lower()
        if confirm == 'n' or confirm == 'cancel':
            print("Deletion cancelled.")
            return False
        if confirm == 'y':
            break
        print("Please enter 'y', 'n', or 'cancel'.")

    # Delete transaction
    self.transactions.remove(transaction)
    print(f"Transaction {transaction_id} deleted successfully!")
    return True


**Notes**:  
Can I automate the tests with a single test file by simulating user inputs?

Yes, yes I can! Added `test_finance_utils.py` for current and future tests. It's incomplete, but I can continue working on it as I progress.

Onto Task 4.

**5/23/2025**: Starting Task 4 

During the meetup last night it was mentioned that we should *not* be using Copilot to write our code. I use a toolkit of AIs that replace my old ways of coding..  

> **TASK -> ROADBLOCK -> RESEARCH (GOOGLE, STACK OVERFLOW) -> REFACTOR -> SUCCESS**

With AI tools, it looks more like this..

> **TASK -> ROADBLOCK -> RESEARCH (GROK, CoPilot) -> REFACTOR -> SUCCESS**

The benefits outweigh the pitfalls IMHO. I write my own code, collaborating with AI to gain insights and follow best practices. For learners, these tools can dramatically enhance the experience when used responsibly and ethically.  

That said, whenever it comes to a task requiring deep understanding of a subject (i.e., python fundamentals), let me emphasize:  
**AI Tools must be combined with mindful coding practices and a goal of learning the subject.** 

Yes, these tools will code for you and there will be times when that is practical. No, this isn't the time. Moving on!  

---

## Analyzing Financial Data

"Weeks 5 and 7 introduced dictionaries and data processing. Calculate metrics like total credits, debits, and transfers, and group by type or customer ID. Use Week 2’s loops, Week 1’s arithmetic, and Week 5’s dictionaries."  

**Code Hint**:  
```python
def analyze_finances(transactions):
    """Calculate and display financial summaries."""
    # Sum credits, debits, transfers
    # Group by type or customer_id
    # Print results
    pass
```

**Requirements**:  
- Sum amounts by type using a dictionary.
- Format numbers with `f"${value:.2f}`.
- Handle transfers separately (niether income nor expense).
- Show the customer with the highest debt amount.
- Calculate percentage of total amount by type.
- Analyze transactions from 2022 only.
- Save analysis to `analysis.txt`.

**Expected Output**:
```
Financial Summary:
Total Credits: $6478.39
Total Debits: $7969.68
Total Transfers: $0.00
Net Balance: $-1491.29
By Type:
  Credit: $6478.39
  Debit: $7969.68
```

Take-Away: 
The analysis should produce a formatted summary, get percentage totals by type, filter by year, and save the analysis to a file.  
Then, *for practice and fun* I can produce a custom analysis that shows more detailed insights. 

**Note**:  
I'm still working with `csv_faker.py` to play and test with a `developer_transactions.csv` file that I use interchangably throughout this exercise.

In [ ]:
# Class method

def analyze_transactions(self):
    """
    Analyze transactions and print summary stats. 
    
    Save the analysis to analysis.txt
    """

    # Check if transactions exist
    if not self.transactions:
        print("No transactions to analyze.")
        return False
    
    # Initialize analysis data
    type_sums = {"debit": 0.0, "credit": 0.0}
    transfer_total = 0.0

    # Process transactions
    for transaction in self.transactions:
        if transaction['type'] == 'debit':
            type_sums['debit'] += abs(transaction['amount'])
        elif transaction['type'] == 'credit':
            type_sums['credit'] += abs(transaction['amount'])
        elif transaction['type'] == 'transfer':
            transfer_total += abs(transaction['amount'])

    # Calculate totals
    total_transactions = len(self.transactions)
    total_debit = type_sums['debit']
    total_credit = type_sums['credit']
    total_transfer = transfer_total
    net_balance = total_credit + total_debit

    # Print summary
    print("\nFinancial Summary:")
    print(f"Total Credits: ${total_credit:,.2f}")
    print(f"Total Debits: ${total_debit:,.2f}")
    print(f"Total Transfers: ${total_transfer:,.2f}")
    print(f"Net Balance: ${net_balance:,.2f}")
    print(f"By type: ")
    for t in type_sums:
        print(f"  {t.capitalize()}: ${type_sums[t]:,.2f}")

    # Save analysis to file
    try:
        with open('analysis.txt', 'w', encoding='utf-8') as file:
            file.write("Financial Summary:\n")
            file.write(f"Total Credits: ${total_credit:,.2f}\n")
            file.write(f"Total Debits: ${total_debit:,.2f}\n")
            file.write(f"Total Transfers: ${total_transfer:,.2f}\n")
            file.write(f"Net Balance: ${net_balance:,.2f}\n")
            file.write("By type:\n")
            for t in type_sums:
                file.write(f"  {t.capitalize()}: ${type_sums[t]:,.2f}\n")
        print("Analysis saved to 'analysis.txt'.")
    except IOError as e:
        self.logger.error(f"Failed to save analysis: {e}")
        print(f"Error: Failed to save analysis to 'analysis.txt': {e}")

    return True

## Saving Transactions & Generating Reports

**Task 5**:  
- Save in-memory changes to the working CSV file (backup created during load).  
- Generate a financial report and save it to a file `report.txt`.  
- Debug outputs and catch exceptions.  

**Example Code**:
```python
def save_transactions(transactions, filename='financial_transactions.csv'):
    """Save transactions to a CSV file."""
    # Open file for writing
    # Write header
    # Write each transaction
    pass

def generate_report(transactions, filename='report.txt'):
    """Generate a text report of financial summaries."""
    # Calculate metrics
    # Write to file
    pass

```

**Recap**:  
Core logic for transaction management is implemented, menu interface is working, and error reporting looks good. With these tasks checked off the list, I'll review the project and make notes highlighting what's working and what I should wrap-up before submission.

## Wrap Up  

**Review error messages and handling** for conciseness.  
**Reduce code duplication** as a best practice.  

The project mentions an 'intuitive menu' interface. I'll be taking some steps to achieve a quality user experience.  
- Consistent error/info log and console messages.
- Color to differentiate for visual clues.  

**Review class method definitions** and helper methods  
**Highlight areas where the requirements are met** to demonstrate understanding  
**Customer ID Suggestions...** rework this?  

Currently, I have backups (snapshots) being saved, multiple transaction files, reports being generated, and until a moment ago an analysis file being generated. That's MUCHO BLOAT.  

**My Plan**:  
- The backups are kept orderly in /snapshots  
- Revise and update `utils.py` and `main.py` for consistency  
- Update `test_finance_utils.py` to include tests for Project Tasks 1-5
- Consider bonus features and enhancements to exercise new skills  

***To the code!***  

---

In [ ]:
# Notes after manual testing:

# 1. During menu operations the program terminal gets cluttered with messages
#    Fix: Flush the output after each operation

# 2. Adding a transaction prints the details in dictionary format
#    Fix: Format the output to be more user-friendly using tabulate

# 3. Update transaction valid customer ids are showing ids not in the transactions
#    Fix: Filter the customer ids to only show those in the transactions
#    After updating, print the changes to the user using tabulate

# 4. Add info messages to a info.txt file when a transaction is loaded, added, updated, or deleted
#    Fix: Use logging to write to info.txt

# 5. Correct the message after analysis to only read "Analysis complete."
#    Fix: Update the print statement to reflect this

# 6. Colorize some of the output
#    Fix: Use the colorama library to add color to the output

# 7. Add a progress bar when loading, analyzing, saving transactions, and generating reports
#    Fix: Build a progress bar from scratch using a simple loop

# 8. If the CSV file is empty, the program should not crash
#    Fix: Add a check for an empty file before processing

# 9. Enhance report generation to include a breakdown of years and quarters
#    Fix: Use the pandas library to generate a more detailed report

# Code examples below:

In [ ]:
# Flush the output
def clear_terminal(self):
    """Clear the terminal screen for a cleaner user interface."""
    try:
        # Use 'cls' for Windows, 'clear' for Unix-based systems
        os.system('cls' if os.name == 'nt' else 'clear')
    except Exception as e:
        self.logger.error(f"Failed to clear terminal: {e}")

In [ ]:
# Format output for adding, updating, and deleting transactions
# to be readable and consistent

# Format and display transaction using tabulate
table_data = [[
    transaction['transaction_id'],
    transaction['date'].strftime('%b %d, %Y'),
    transaction['customer_id'],
    f"${abs(transaction['amount']):,.2f}{' (Debit)' if transaction['amount'] < 0 else ''}",
    transaction['type'].capitalize(),
    transaction['description'][:30] + ('...' if len(transaction['description']) > 30 else '')
]]
headers = ['ID', 'Date', 'Customer', 'Amount', 'Type', 'Description']
print("\nTransaction added successfully:")
print(tabulate(table_data, headers=headers, tablefmt='grid'))


In [ ]:
# Update customer ID input with suggestions from most recent transactions
# Sort transactions by date (descending) and get unique customer IDs

recent_transactions = sorted(self.transactions, key=lambda t: t['date'], reverse=True)
seen_ids = set()
customer_ids = []
for t in recent_transactions:
    if t['customer_id'] > 0 and t['customer_id'] not in seen_ids:
        customer_ids.append(t['customer_id'])
        seen_ids.add(t['customer_id'])
        if len(customer_ids) >= 10:  # Limit to 10 IDs
            break
if customer_ids:
    print(f"Recent customer IDs: {', '.join(map(str, customer_ids))}{'...' if len(seen_ids) < len(set(t['customer_id'] for t in self.transactions if t['customer_id'] > 0)) else ''}")
else:
    print("No customer IDs available.")

In [ ]:
# Log activity for auditing and testing to activity.txt
# Tracks: loading and backup transactions, adding, updating, deleting transactions

def __init__(self):
    """Initialize transactions list and configure logging."""
    self.transactions = []
    # Configure logging with handlers for errors and activity
    self.logger = logging.getLogger('FinanceUtils')
    self.logger.setLevel(logging.INFO)  # Set to INFO to capture both INFO and ERROR
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    # Error handler for errors.txt
    try:
        self.error_handler = logging.FileHandler('errors.txt', mode='a', encoding='utf-8')
        self.error_handler.setLevel(logging.ERROR)
        self.error_handler.setFormatter(formatter)
        self.logger.addHandler(self.error_handler)
    except IOError as e:
        print(f"Error: Cannot configure logging to errors.txt: {e}")
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.ERROR)
        console_handler.setFormatter(formatter)
        self.logger.addHandler(console_handler)
        self.error_handler = None

    # Info handler for activity.txt
    try:
        self.activity_handler = logging.FileHandler('activity.txt', mode='a', encoding='utf-8')
        self.activity_handler.setLevel(logging.INFO)
        self.activity_handler.setFormatter(formatter)
        self.logger.addHandler(self.activity_handler)
    except IOError as e:
        print(f"Error: Cannot configure logging to activity.txt: {e}")
        if not self.error_handler:  # Only add console handler if no error handler exists
            console_handler = logging.StreamHandler()
            console_handler.setLevel(logging.INFO)
            console_handler.setFormatter(formatter)
            self.logger.addHandler(console_handler)
        self.activity_handler = None

In [ ]:
# Colorize output using colorama inside utils.py

# Initialize colorama for colored output (optional)
try:
    from colorama import init, Fore, Style
    init(autoreset=True)  # Auto-reset colors after each print
    self.color = {
        'cyan': Fore.CYAN,
        'green': Fore.GREEN,
        'yellow': Fore.YELLOW,
        'red': Fore.RED,
        'reset': Style.RESET_ALL
    }
except ImportError:
    self.logger.info("Colorama not installed; using plain text output.")
    self.logger.info("For a more visual experience, consider installing colorama.")
    self.color = {'cyan': '', 'green': '', 'yellow': '', 'red': '', 'reset': ''}  # Fallback to empty strings

In [ ]:
# Add a retro progress bar for loading and saving transactions, and generating reports
# This is a fun text-based progress bar that uses asterisks to indicate *actual* progress

# Helper method to display a retro-style asterisk progress bar.
def _display_progress_bar(self, progress, total, prefix="Processing"):
    """Display a retro-style asterisk progress bar."""
    try:
        import sys
        import math
        # Calculate percentage and number of asterisks (max 5)
        percent = progress / total if total > 0 else 1.0
        asterisks = min(5, math.ceil(percent * 5))
        spaces = 5 - asterisks
        bar = f"[{self.color['yellow']}{'*' * asterisks}{' ' * spaces}{self.color['reset']}]"
        # Print in place with carriage return
        sys.stdout.write(f"\r{prefix}: {bar} {int(percent * 100)}%")
        sys.stdout.flush()
    except Exception as e:
        self.logger.error(f"Failed to display progress bar: {e}")
        # Fallback to plain text
        print(f"{prefix}: {int((progress / total) * 100 if total > 0 else 100)}%")


In [ ]:
# Add checks for empty CSV file - also added info messages to class methods

if finance.transactions:
    filter_type = input("Enter type to filter (credit/debit/transfer, or press Enter for all): ").strip()
    if not filter_type:
        filter_type = None
    filter_year = input("Enter year to filter (e.g., 2020, or press Enter for all): ").strip()
    if not filter_year:
        filter_year = None
    if not finance.view_transactions(filter_type, filter_year):
        print("No transactions displayed.")
else:
    print("No transactions loaded. Please load transactions first.")

In [ ]:
# Enhance the report generation to include an additional breakdown of years and quarters
# and a detailed temporal analysis.

def generate_report(self, filename='report.txt'):
    """
    Generate a financial report with yearly and quarterly breakdowns, top customers,
    year-over-year growth, and anomaly detection, saving it to a text file.

    Summary includes:
    - Date range and total transactions
    - Financial summary (credits, debits, transfers, net balance)
    - Breakdown by type (count and percentage)
    - Yearly and quarterly breakdowns (credits, debits, transfers, net balance, counts)
    - Top 5 customers by transaction volume
    - Year-over-year growth for credits, debits, and net balance
    - Anomalous transactions (amounts > 3 standard deviations from mean)

    Args:
        filename (str): Path to the report file.
        
    Returns:
        bool: True if report generation succeeds, False otherwise.
    """
    if not self.transactions:
        self.logger.info("Attempted to generate report with no transactions loaded")
        print(f"{self.color['red']}No transactions loaded. Please load a transaction file first.{self.color['reset']}")
        return False
    
    try:
        # Add timestamp to filename
        timestamp = datetime.now().strftime('%Y%m%d')
        filename = f"report_{timestamp}.txt"

        # Define stages for progress
        stages = 8  # Date range, totals, type breakdown, yearly, quarterly, top customers, YoY, anomalies
        current_stage = 0

        with open(filename, 'w', encoding='utf-8') as file:
            file.write("Financial Report\n")
            file.write("=================\n\n")

            # Date range and total transactions
            dates = [t['date'] for t in self.transactions]
            min_date = min(dates).strftime('%Y-%m-%d')
            max_date = max(dates).strftime('%Y-%m-%d')
            file.write(f"Date Range: {min_date} to {max_date}\n")
            file.write(f"Total Transactions: {len(self.transactions):,}\n")
            file.write("\n")
            current_stage += 1
            self._display_progress_bar(current_stage, stages, "Generating Report")

            # Financial summary
            total_credit = sum(t['amount'] for t in self.transactions if t['type'] == 'credit')
            total_debit = sum(abs(t['amount']) for t in self.transactions if t['type'] == 'debit')
            total_transfer = sum(abs(t['amount']) for t in self.transactions if t['type'] == 'transfer')
            net_balance = total_credit - total_debit
            file.write("Financial Summary:\n")
            file.write(f"  Total Credits: ${total_credit:,.2f}\n")
            file.write(f"  Total Debits: ${total_debit:,.2f}\n")
            file.write(f"  Total Transfers: ${total_transfer:,.2f}\n")
            file.write(f"  Net Balance: ${net_balance:,.2f}\n")
            file.write("\n")
            current_stage += 1
            self._display_progress_bar(current_stage, stages, "Generating Report")

            # Breakdown by type
            total_transactions = len(self.transactions)
            credit_count = sum(1 for t in self.transactions if t['type'] == 'credit')
            debit_count = sum(1 for t in self.transactions if t['type'] == 'debit')
            transfer_count = sum(1 for t in self.transactions if t['type'] == 'transfer')
            file.write("Breakdown by Type:\n")
            if total_transactions > 0:
                credit_percentage = (credit_count / total_transactions) * 100
                debit_percentage = (debit_count / total_transactions) * 100
                transfer_percentage = (transfer_count / total_transactions) * 100
                file.write(f"  Credit: {credit_count:,} transactions ({credit_percentage:.2f}%)\n")
                file.write(f"  Debit: {debit_count:,} transactions ({debit_percentage:.2f}%)\n")
                file.write(f"  Transfer: {transfer_count:,} transactions ({transfer_percentage:.2f}%)\n")
            else:
                file.write("  No transactions to analyze.\n")
            file.write("\n")
            current_stage += 1
            self._display_progress_bar(current_stage, stages, "Generating Report")

            # Yearly and quarterly breakdown
            yearly_data = {}
            for t in self.transactions:
                year = t['date'].year
                quarter = (t['date'].month - 1) // 3 + 1  # Q1: Jan-Mar, Q2: Apr-Jun, etc.
                if year not in yearly_data:
                    yearly_data[year] = {
                        'credits': 0.0, 'debits': 0.0, 'transfers': 0.0, 'count': 0,
                        'quarters': {1: {'credits': 0.0, 'debits': 0.0, 'transfers': 0.0, 'count': 0},
                                     2: {'credits': 0.0, 'debits': 0.0, 'transfers': 0.0, 'count': 0},
                                     3: {'credits': 0.0, 'debits': 0.0, 'transfers': 0.0, 'count': 0},
                                     4: {'credits': 0.0, 'debits': 0.0, 'transfers': 0.0, 'count': 0}}
                    }
                if t['type'] == 'credit':
                    yearly_data[year]['credits'] += t['amount']
                    yearly_data[year]['quarters'][quarter]['credits'] += t['amount']
                elif t['type'] == 'debit':
                    yearly_data[year]['debits'] += abs(t['amount'])
                    yearly_data[year]['quarters'][quarter]['debits'] += abs(t['amount'])
                elif t['type'] == 'transfer':
                    yearly_data[year]['transfers'] += abs(t['amount'])
                    yearly_data[year]['quarters'][quarter]['transfers'] += abs(t['amount'])
                yearly_data[year]['count'] += 1
                yearly_data[year]['quarters'][quarter]['count'] += 1
            file.write("Breakdown by Year and Quarter:\n")
            for year in sorted(yearly_data.keys()):
                data = yearly_data[year]
                net = data['credits'] - data['debits']
                file.write(f"  {year}:\n")
                file.write(f"    Total Credits: ${data['credits']:,.2f}\n")
                file.write(f"    Total Debits: ${data['debits']:,.2f}\n")
                file.write(f"    Total Transfers: ${data['transfers']:,.2f}\n")
                file.write(f"    Net Balance: ${net:,.2f}\n")
                file.write(f"    Transactions: {data['count']:,}\n")
                for q in range(1, 5):
                    qdata = data['quarters'][q]
                    if qdata['count'] > 0:
                        qnet = qdata['credits'] - qdata['debits']
                        file.write(f"    Q{q} (Jan-Mar {year}):\n" if q == 1 else
                                   f"    Q{q} (Apr-Jun {year}):\n" if q == 2 else
                                   f"    Q{q} (Jul-Sep {year}):\n" if q == 3 else
                                   f"    Q{q} (Oct-Dec {year}):\n")
                        file.write(f"      Credits: ${qdata['credits']:,.2f}\n")
                        file.write(f"      Debits: ${qdata['debits']:,.2f}\n")
                        file.write(f"      Transfers: ${qdata['transfers']:,.2f}\n")
                        file.write(f"      Net Balance: ${qnet:,.2f}\n")
                        file.write(f"      Transactions: {qdata['count']:,}\n")
            file.write("\n")
            current_stage += 1
            self._display_progress_bar(current_stage, stages, "Generating Report")

            # Top 5 customers by transaction volume
            customer_totals = {}
            for t in self.transactions:
                cid = t['customer_id']
                amount = abs(t['amount'])
                customer_totals[cid] = customer_totals.get(cid, 0.0) + amount
            top_customers = sorted(customer_totals.items(), key=lambda x: x[1], reverse=True)[:5]
            file.write("Top 5 Customers by Transaction Volume:\n")
            for cid, total in top_customers:
                file.write(f"  Customer ID {cid}: ${total:,.2f}\n")
            file.write("\n")
            current_stage += 1
            self._display_progress_bar(current_stage, stages, "Generating Report")

            # Year-over-year growth
            file.write("Year-over-Year Growth:\n")
            years = sorted(yearly_data.keys())
            for i in range(1, len(years)):
                prev_year = years[i-1]
                curr_year = years[i]
                prev_data = yearly_data[prev_year]
                curr_data = yearly_data[curr_year]
                credit_growth = ((curr_data['credits'] - prev_data['credits']) / prev_data['credits'] * 100) if prev_data['credits'] != 0 else 0.0
                debit_growth = ((curr_data['debits'] - prev_data['debits']) / prev_data['debits'] * 100) if prev_data['debits'] != 0 else 0
                net_prev = prev_data['credits'] - prev_data['debits']
                net_curr = curr_data['credits'] - curr_data['debits']
                net_growth = ((net_curr - net_prev) / net_prev * 100) if net_prev != 0 else 0.0
                file.write(f"  {curr_year} vs {prev_year}:\n")
                file.write(f"    Credits: {credit_growth:+.2f}%\n")
                file.write(f"    Debits: {debit_growth:+.2f}%\n")
                file.write(f"    Net Balance: {net_growth:+.2f}%\n")
            file.write("\n")
            current_stage += 1
            self._display_progress_bar(current_stage, stages, "Generating Report")

            # Anomaly detection (transactions > 3 std deviations from mean)
            amounts = [abs(t['amount']) for t in self.transactions]
            if amounts:
                mean = sum(amounts) / len(amounts)
                variance = sum((x - mean) ** 2 for x in amounts) / len(amounts)
                std_dev = variance ** 0.5
                threshold = mean + 3 * std_dev
                anomalies = [(t['transaction_id'], t['amount'], t['date'].strftime('%Y-%m-%d'), t['customer_id'])
                             for t in self.transactions if abs(t['amount']) > threshold]
                file.write("Anomalous Transactions (> 3 std dev from mean amount):\n")
                if anomalies:
                    for tid, amount, date, cid in anomalies:
                        file.write(f"  ID {tid}: ${amount:,.2f} on {date} (Customer {cid})\n")
                else:
                    file.write("  No anomalies detected.\n")
            else:
                file.write("  No transactions to analyze for anomalies.\n")
            current_stage += 1
            self._display_progress_bar(current_stage, stages, "Generating Report")

        # Final progress update and success message
        current_stage += 1
        self._display_progress_bar(current_stage, stages, "Generating Report")
        print()  # Newline after progress bar
        print(f"{self.color['green']}Report generated and saved to '{filename}'.{self.color['reset']}")
        self.logger.info(f"Generated report: '{filename}'")
        return True
    
    except IOError as e:
        self.logger.error(f"Failed to generate report: {e}")
        print(f"{self.color['red']}Error: Failed to generate report '{filename}': {self.color['reset']}{e}")
        return False

## Ending Thoughts  

Hi! This was a challenging project that ending up being fun.   

My goal was to get the basic requirements first, then consider adding bonus features while striving to keep from using external libraries. Tabulate was an easy choice for prettifying the table views, and coloroma more of a personal choice (I made it optional) for visual performance. Overall they don't add much overhead to the project considering it's purpose, but injecting some fun did make the process more enjoyable.  

I have some previous experience with python and developing this project definitely refreshed the fundamentals and basics for me. 

Below is a comparison evaluating my current implementation against the original project requirements.   

---

## Project Requirements Alignment

| Requirement | Description | Status | Implementation Details | Notes |
|-------------|-------------|--------|------------------------|-------|
| Load Transactions | Load transactions from a CSV file (`financial_transactions.csv`) with fields: `transaction_id`, `date`, `customer_id`, `amount`, `type`, `description`. | ✅ Met | `load_transactions` reads CSV, validates fields, handles errors (e.g., missing columns, invalid data), and creates backups in `snapshots/`. Progress bar shows loading progress. | Robust error handling (e.g., `FileNotFoundError`, `UnicodeDecodeError`) and logging to `errors.txt` and `activity.txt`. |
| Add Transactions | Allow users to add new transactions with input validation and unique `transaction_id`. | ✅ Met | `add_transaction` prompts for date, customer ID (with recent ID suggestions), amount, type, and description. Validates inputs and logs to `activity.txt`. | Customer ID suggestions enhance usability. Works without loaded transactions, as intended. |
| View Transactions | Display transactions with optional filters (type, year) and pagination. | ✅ Met | `view_transactions` supports filters for `type` (credit/debit/transfer) and `year`, with 10-transaction pages and navigation (start/next/prev/end/exit). Uses `tabulate` with colored headers. | Checks for empty transactions in `main.py`, with consistent error message and logging. |
| Update Transactions | Update existing transactions by ID, modifying any field. | ✅ Met | `update_transaction` prompts for ID, displays current details, and allows updates to date, customer ID, amount, type, description. Validates inputs and logs to `activity.txt`. | Checks for empty transactions, provides customer ID suggestions, and uses colored output. |
| Delete Transactions | Delete transactions by ID with confirmation. | ✅ Met | `delete_transaction` prompts for ID, displays transaction, requires confirmation (yes/no), and removes it. Logs to `activity.txt`. | Checks for empty transactions, uses colored confirmation prompt and success message. |
| Analyze Transactions | Provide summary statistics (e.g., total credits, debits, transfers, net balance). | ✅ Met | `analyze_transactions` computes totals for credits, debits, transfers, and net balance, displaying with colored output. | Checks for empty transactions, suitable for quick console-based analysis. |
| Save Transactions | Save transactions to a CSV file, preserving format. | ✅ Met | `save_transactions` writes transactions to CSV, maintaining field structure. Progress bar shows saving progress, logs to `activity.txt`. | Checks for empty transactions, handles `IOError` with logging. |
| Generate Report | Produce a detailed financial report saved to a text file. | ✅ Exceeded | `generate_report` includes date range, totals, type breakdown, yearly/quarterly breakdowns, top 5 customers, year-over-year growth, and anomaly detection. Progress bar and colored success message. | Enhanced beyond basic requirements with research-driven features (e.g., YoY growth, anomalies). |
| User Interface | Provide a clear, interactive CLI with a menu-driven interface. | ✅ Met | `main.py` offers a 9-option menu with clear prompts, colored output (cyan header, green success, red errors), and `clear_terminal` for clarity. | Streamlined checks for empty transactions improve usability. |
| Input Validation | Validate all user inputs (e.g., dates, IDs, amounts, types). | ✅ Met | All methods (`add_transaction`, `update_transaction`, `view_transactions`, etc.) validate inputs (e.g., `YYYY-MM-DD` dates, positive integers, valid types). Errors logged to `errors.txt`. | Consistent error messages with red color enhance feedback. |
| Error Handling | Handle file I/O errors, invalid data, and edge cases gracefully. | ✅ Met | Methods handle `IOError`, `csv.Error`, `UnicodeDecodeError`, `ValueError`, etc., with logging to `errors.txt` and user-friendly messages. | Fallbacks (e.g., console logging, plain text for `colorama`) ensure robustness. |
| Logging | Log errors and key operations for debugging and auditing. | ✅ Exceeded | Errors logged to `errors.txt`, successful operations (load, save, add, update, delete, report) and empty transaction attempts to `activity.txt`. | Comprehensive logging supports traceability and debugging. |
| Backup System | Create timestamped backups of the transaction file. | ✅ Met | `load_transactions` creates backups in `snapshots/` with timestamps (e.g., `backup_20250525_140300.csv`). | Logged to `activity.txt`, with error handling for backup failures. |
| Performance | Handle large datasets (e.g., 100,001 transactions) efficiently. | ✅ Met | All operations (loading, saving, reporting) are O(n) or better, tested with 100,001 transactions. Progress bars provide feedback for long operations. | Minimal memory usage with in-memory `self.transactions` list. |
| Extensibility | Design code to allow future enhancements (e.g., new report features). | ✅ Met | Modular `FinanceUtils` class, clean method structure, and helper methods (e.g., `_display_progress_bar`) support additions like quarterly breakdowns and anomalies. | Recent revisions (e.g., report enhancements) demonstrate extensibility. |
| Optional Enhancements | Support visual improvements (e.g., colors, progress bars). | ✅ Exceeded | `colorama` for colored output, retro asterisk progress bar for load/save/report, and enhanced report features (e.g., YoY growth). | Optional `colorama` with fallback ensures minimal impact. |   

---  

We'll be moving fast in future exercises, so feedback is much welcomed!! It really helps me improve and is always appreciated.  
Thanks for checking out this project, contact me at [prizecoffeecup@gmail.com](mailto:prizecoffeecup@gmail.com) 🚀